In [28]:
# linux or mac
import os 
os.environ['WATTTIME_USER'] = 'nomanbashir'
os.environ['WATTTIME_PASSWORD'] = 'vikner-bozda4-fesrEb'

import pandas as pd
from tqdm import tqdm

In [16]:
from watttime import WattTimeMyAccess
from watttime import WattTimeForecast
from watttime import WattTimeHistorical

wt_forecast = WattTimeForecast()

wt_historical = WattTimeHistorical()

wt_myaccess = WattTimeMyAccess()

In [17]:
regions_available = wt_myaccess.get_access_pandas()
# regions_available.to_csv('regions_available.csv', index=False)
print(regions_available)

       signal_type       region                       region_name  \
0         co2_moer         AECI      Associated Electric Coop Inc   
1         co2_moer         AECI      Associated Electric Coop Inc   
2         co2_moer         AESO  Alberta Electric System Operator   
3         co2_moer         AESO  Alberta Electric System Operator   
4         co2_moer           AT                           Austria   
..             ...          ...                               ...   
526       co2_moer          WEM                 Western Australia   
527       co2_moer          WEM                 Western Australia   
528  health_damage  CAISO_NORTH           California ISO Northern   
529  health_damage  CAISO_NORTH           California ISO Northern   
530  health_damage  CAISO_NORTH           California ISO Northern   

          endpoint       model data_start train_start  train_end  \
0    v3/historical  2022-10-01 2020-01-01  2020-03-01 2022-03-01   
1      v3/forecast  2022-10-01 2021

In [14]:
hist_forecasts_hour = wt_forecast.get_historical_forecast_pandas(
    start = '2023-12-01 00:00+00:00',
    end = '2023-12-01 00:59+00:00',
    region = 'CAISO_NORTH',
    signal_type = 'co2_moer',
)
avg_forecast = hist_forecasts_hour['value'].mean()

# hist_forecasts.to_csv('hist_forecasts.csv', index=False)

print(avg_forecast)

933.8888599537037


In [21]:
hist_hour = wt_historical.get_historical_pandas(
    start = '2023-12-01 00:00+00:00',
    end = '2023-12-01 00:59+00:00',
    region = 'CAISO_NORTH',
    signal_type = 'co2_moer',
)
avg_marginal = hist_hour['value'].mean()

print(avg_marginal)

1023.6666666666666


In [45]:
carbon_data = [
    # ("ap-southeast-2", "NEM_NSW"),
    # ("ca-central-1", "HQ"),
    ("eu-central-1", "DE"),
    # ("eu-north-1", "SE"),
    # ("eu-west-2", "UK"),
    # ("eu-west-3", "FR"),
    # ("us-east-1", "PJM_DC"),
    # ("us-west-2", "PACW"),
    # ("us-west-1", "CAISO_NORTH"),
]

for csv_name, region in carbon_data:
    # load csv from carbon-data
    df = pd.read_csv(f"carbon-data/{csv_name}.csv", parse_dates=["datetime"])

    # filter data to only consider 2022 data
    df = df[df['datetime'] >= pd.Timestamp("2022-01-01", tz='UTC')]
    
    # get the list of timestamps in the underlying data
    timestamps = df['datetime']
    
    hist_marginal = []
    hist_marginal_forecast = []
    for time in tqdm(timestamps):
        start_time = time
        end_time = time + pd.Timedelta('59 minutes')

        model = "2022-10-01"
        # if start_time < pd.Timestamp("2020-03-01", tz='UTC'):
        #     model = "2022-10-01"

        # get the data from WattTime
        try:
            hist_hour = wt_historical.get_historical_pandas(
                start = start_time,
                end = end_time,
                region = region,
                signal_type = 'co2_moer',
                # model = model,
            )
            hist_forecasts_hour = wt_forecast.get_historical_forecast_pandas(
                start = start_time,
                end = end_time,
                region = region,
                signal_type = 'co2_moer',
                model = model,
            )
        except:
            # if it fails, wait a minute
            time.sleep(60)
            # try again
            hist_hour = wt_historical.get_historical_pandas(
                start = start_time,
                end = end_time,
                region = region,
                signal_type = 'co2_moer',
                # model = model,
            )
            hist_forecasts_hour = wt_forecast.get_historical_forecast_pandas(
                start = start_time,
                end = end_time,
                region = region,
                signal_type = 'co2_moer',
                model = model,
            )
        avg_marginal = hist_hour['value'].mean()
        avg_forecast = hist_forecasts_hour['value'].mean()
        hist_marginal.append(avg_marginal)
        hist_marginal_forecast.append(avg_forecast)
    df['marginal_carbon_avg'] = hist_marginal
    df['marginal_forecast_avg'] = hist_marginal_forecast
    df.to_csv(f"marginal-data/{csv_name}.csv", index=False)
    print(f"Processed {csv_name}")


 19%|█▊        | 1637/8760 [2:11:11<9:22:20,  4.74s/it] 